# Django Views
A view function, or view for short, is a Python function that takes a Web request and returns a Web response. This response can be the HTML contents of a Web page, a redirect, a 404 error, an XML document, an image . . . or anything, really.

Django distinguishes between two main types of views:
1. **Function-Based Views (FBVs)**
2. **Class-Based Views (CBVs)**


## Rendering HTML Templates
The most common task of a view is to load a template, fill it with data (context), and return it as an HTTP response. Django provides shortcuts to make this easy.

### 1. Using `render()` shortcut (Best Practice)
The `render()` function is a wrapper that commonly combines a template, a context dictionary, and an HTTP request object into a fully rendered `HttpResponse`.

```python
from django.shortcuts import render
from .models import Article

def article_list(request):
    # 1. Fetch data from database
    articles = Article.objects.all()
    
    # 2. Define the context (variables available in template)
    context = {
        'articles': articles,
        'page_title': 'Latest News'
    }
    
    # 3. Render the template with the context
    return render(request, 'news/article_list.html', context)
```

### 2. Using `TemplateView` (Class-Based Shortcut)
If you just need to render a static page (like an 'About Us' page) or a simple data page, `TemplateView` is the cleanest approach.

```python
from django.views.generic import TemplateView

class AboutPageView(TemplateView):
    template_name = "about.html"
    
    def get_context_data(self, **kwargs):
        context = super().get_context_data(**kwargs)
        context['team_name'] = "The Avengers"
        return context
```
**URL Conf Usage:**
```python
path('about/', AboutPageView.as_view())
```

## Advanced Function-Based Views (FBV)
FBVs are simple Python functions. They are easy to read and understand, making them great for simple logic.

### Basic Syntax
```python
from django.http import HttpResponse
import datetime

def current_datetime(request):
    now = datetime.datetime.now()
    html = "<html><body>It is now %s.</body></html>" % now
    return HttpResponse(html)
```

### Using Decorators
You can enhance FBVs using decorators, for example, to require login.

```python
from django.contrib.auth.decorators import login_required
from django.shortcuts import render

@login_required
def my_protected_view(request):
    return render(request, 'secret_page.html')
```

## Class-Based Views (CBV)
CBVs provide an object-oriented way to organize your view code. They are excellent for reuse and inheritence.

### Basic Syntax (The `View` class)
```python
from django.http import HttpResponse
from django.views import View

class MyView(View):
    def get(self, request):
        return HttpResponse('Result of GET request')

    def post(self, request):
        return HttpResponse('Result of POST request')
```

### Generic Display Views
Django provides "Generic Views" for common tasks like displaying a list of objects or a single object.

#### ListView
```python
from django.views.generic import ListView
from apps.core.models import Article

class ArticleListView(ListView):
    model = Article
    template_name = 'article_list.html'
    context_object_name = 'articles'
    paginate_by = 10
```

#### DetailView
```python
from django.views.generic import DetailView
from apps.core.models import Article

class ArticleDetailView(DetailView):
    model = Article
    template_name = 'article_detail.html'
```

### Using Mixins
Instead of decorators, CBVs use Mixins to enforce permissions. **Mixins must always come first** in the inheritance list.

```python
from django.contrib.auth.mixins import LoginRequiredMixin
from django.views.generic import TemplateView

class SecretView(LoginRequiredMixin, TemplateView):
    template_name = 'secret.html'
    login_url = '/login/'
```

## Response Types
While `HttpResponse` is the standard, Django offers others:

- **JsonResponse**: Returns JSON-encoded data. Essential for AJAX/APIs.
  ```python
  from django.http import JsonResponse
  def api_view(request):
      return JsonResponse({'status': 'ok', 'count': 10})
  ```

- **StreamingHttpResponse**: Used for large files or long-running processes to send data in chunks.
  ```python
  import csv
  from django.http import StreamingHttpResponse

  class Echo:
      def write(self, value): return value

  def streaming_csv_view(request):
      rows = (["Row {}".format(idx), str(idx)] for idx in range(1000))
      pseudo_buffer = Echo()
      writer = csv.writer(pseudo_buffer)
      response = StreamingHttpResponse((writer.writerow(row) for row in rows),
                                       content_type="text/csv")
      response['Content-Disposition'] = 'attachment; filename="somefilename.csv"'
      return response
  ```